In [ ]:
# FIAP - Curso IA para Devs
# Tech Challenge 04 
# O PROBLEMA: 
# --- O Tech Challenge desta fase será a criação de uma aplicação que utilize análise de vídeo. O seu projeto deve incorporar as técnicas de reconhecimento
# --- facial, análise de expressões emocionais em vídeos e detecção de atividades.
# A PROPOSTA DO DESAFIO
# --- Você deverá criar uma aplicação a partir do vídeo que se encontra disponível na plataforma do aluno, e que execute as seguintes tarefas:
# --- 1. Reconhecimento facial: Identifique e marque os rostos presentes no vídeo.
# --- 2. Análise de expressões emocionais: Analise as expressões emocionais dos rostos identificados.
# --- 3. Detecção de atividades: Detecte e categorize as atividades sendo realizadas no vídeo.
# --- 4. Geração de resumo: Crie um resumo automático das principais atividades e emoções detectadas no vídeo.
#
# >>> *** Este Jupyter Notebook já é o relatório com as informações exigidas no desafio ao final do arquivo
# >>> *** O Vídeo gravado pelo grupo está disponível na URL: https://XXXXXXXXXXXXX.YYYYY
#
# Grupo 44
# Francisco Antonio Guilherme
# fagn2013@gmail.com

# Marcelo Lima Gomes
# marcelolimagomes@gmail.com

# FELIPE MORAES DOS SANTOS
# felipe.moraes.santos2014@gmail.com

## Configuração do ambiente de execução python. Deve ser executado uma única vez.
### Caso necessário configurar, remova os marcados de comentários de cada linha.

In [ ]:
## Configurando python environment. Necessário ter instalado uma GPU Nvdia (no meu caso RTX-3060) + Anaconda
# conda create -n tc5 -c rapidsai -c conda-forge -c nvidia rapids=24.2 python=3.10 'cuda-version>=12.0,<=12.5' tensorflow[and-cuda]==2.15.0 'pytorch=*=*cuda*' torchvision deepface
# pip install opencv-python cvzone tqdm mediapipe

## >>>Baixa classificador para o Mediapipe
# !wget -O classifier.tflite -q https://storage.googleapis.com/mediapipe-models/image_classifier/efficientnet_lite2/float32/latest/efficientnet_lite2.tflite

## Importando bibliotecas necessárias

In [ ]:
import warnings
import os

import cv2
import cvzone
from deepface import DeepFace
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python.components import processors
from mediapipe.tasks.python import vision
from tqdm import tqdm
from ultralytics import YOLO
import pandas as pd

# Configuração de exibição dos resultados utilizando Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Supressão de warnings
warnings.filterwarnings('ignore')

# Configuração para multithread ao utilizar a biblioteca YOLO
os.environ['OMP_NUM_THREADS'] = '20'
os.environ['NUMEXPR_MAX_THREADS'] = '20'

2024-11-16 19:10:20.711196: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-16 19:10:20.732618: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 19:10:20.732635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 19:10:20.733115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 19:10:20.736775: I tensorflow/core/platform/cpu_feature_guar

## Instanciando modelos de classificação de atividades

In [4]:
# Instanciando o classificador utilizando o modelo YOLO v11
yolo_class = YOLO('yolo11x-cls.pt')

# Instanciando o classificador utilizando a api Mediapipe
VisionRunningMode = mp.tasks.vision.RunningMode
base_options = python.BaseOptions(model_asset_path='classifier.tflite')
options = vision.ImageClassifierOptions(
    base_options=base_options,
    max_results=1,
    running_mode=VisionRunningMode.VIDEO,
    display_names_locale='pt')
classifier = vision.ImageClassifier.create_from_options(options)

libEGL warning: DRI3: Screen seems not DRI3 capable
libEGL warning: DRI2: failed to authenticate
libEGL warning: DRI3: Screen seems not DRI3 capable
I0000 00:00:1731795023.745052  854829 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1731795023.749263  854984 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.2), renderer: llvmpipe (LLVM 17.0.6, 256 bits)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1731795023.766139  854987 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


## Configurando analisador do vídeo

In [ ]:
video_path = './data/video.mp4' # Configurando arquivo de entrada para o algorimto
output_video_path = './data/video_result.mp4' # Configurando arquivo de saída para o algorimto

# Abrindo o arquivo de entrada
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
  print(f"Não foi possível abrir o arquivo de vídeo: {video_path}")

# Configuração de frequência de aplicação dos modelos de predição. 
# Ex. 1 = aplica a cada 1 frame; 5 = aplica a cada 5 frames
frames_scape = 1
count = 1

# Extraíndo propriedades do vídeo
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f'Original: width: {width} - height: {height} - fps: {fps} - total_frames: {total_frames}')

middle = int(width / 4)
bottom = height - 10

# Configurando arquivo de vídeo de saída
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Definição qual modelo para classificação de atividades
class_method = 'MEDIAPIPE' ## MEDIAPIPE / YOLO

Original: width: 1280 - height: 720 - fps: 30.0 - total_frames: 3326


## Executando algoritmos de classificação de Emoções e Atividades

In [6]:
# Variável para armazenar os dados a serem gravados no relatório final
resumo = []

# Loop para processar cada frame do vídeo com barra de progresso
for frame_index in tqdm(range(total_frames), desc="Processando vídeo"):
  # lendo o vídeo frame a frame
  ret, frame = cap.read()

  # Finaliza o loop caso a leitura do frame do vídeo não encontre dados
  if not ret:
    print("Can't receive frame (stream end?). Exiting ...")
    break

  # Calculando o timestamp do frame corrente
  frame_timestamp_ms = int(1000 * frame_index / fps)

  # Condição para validar se deve ser executado a predição com base na informação da variável "frames_scape"
  if count == frames_scape:
    count = 1
    
    # Executando algoritmo de detecção de rostos
    rostos_detectados = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False, detector_backend='yolov8')
    
    # Executando algoritmo de classificação de atividades
    if class_method == 'YOLO':
      res = yolo_class.predict(frame, verbose=False)
      class_name = res[0].names[res[0].probs.top1]
      class_score = res[0].probs.top1conf
    elif class_method == 'MEDIAPIPE':
      # Convert the frame received from OpenCV to a MediaPipe’s Image object.
      mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
      
      # Perform image classification on the video frame.
      classification_result = classifier.classify_for_video(mp_image, frame_timestamp_ms)

      # Process the classification result. In this case, visualize it.
      top_category = classification_result.classifications[0].categories[0]
      class_name = top_category.category_name
      class_score = top_category.score
    else:
      raise Exception(f"Classificador não encontrado: {class_method}")
      
    # Escreve no vídeo a atividade predita e o percentual de precisão (score)
    cv2.putText(frame, f"Activity: {class_name} ({class_score * 100:.2f}%)", (middle, bottom), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Desenhando retangulos nas faces detectadas e printando o label com a emoção predita
    list_emotions = []
    for face in rostos_detectados:
      x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
      cvzone.cornerRect(frame, [x, y, w, h], l=9, t=2, rt=1)
      dominant_emotion = face['dominant_emotion']
      list_emotions.append(dominant_emotion)
      cv2.putText(frame, dominant_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Armazenando dados para gerar relatório final
    if len(list_emotions) > 0:
      for emotion in list_emotions:
        resumo.append({"frame_index": frame_index,
                      "total_frames": total_frames,
                      "activity": class_name,
                      "score": class_score,
                      "faces_count": len(rostos_detectados),
                      "emotion": emotion})
    else:
      resumo.append({"frame_index": frame_index,
                    "total_frames": total_frames,
                    "activity": class_name,
                    "score": class_score,
                    "faces_count": len(rostos_detectados),
                    "emotion": None})
  else:
    count += 1

  # Grava o frame corrente com anotações das predições no arquivo de saída
  out.write(frame)

# Close the window
cap.release()
out.release()
# De-allocate any associated memory usage
cv2.destroyAllWindows()

Processando vídeo: 100%|██████████| 3326/3326 [13:38<00:00,  4.06it/s]


# Relatório: 
## O resumo obtido automaticamente com as principais atividades e emoções detectadas no vídeo. Nesse momento esperando que o relatório inclua:
- Total de frames analisados.
- Número de anomalias detectadas.

In [7]:
df_relatorio_detalhado = pd.DataFrame(resumo)
df2 = pd.DataFrame({"type": ["activity" for i in df_relatorio_detalhado["activity"].value_counts()], "name": df_relatorio_detalhado["activity"].value_counts().index, "count": df_relatorio_detalhado["activity"].value_counts()})
df3 = pd.DataFrame({"type": ["emotion" for i in df_relatorio_detalhado["emotion"].value_counts()], "name": df_relatorio_detalhado["emotion"].value_counts().index, "count": df_relatorio_detalhado["emotion"].value_counts()})
df_relatorio_resumido = pd.concat([df2, df3]).reset_index(drop=True).sort_values(["type", "count"], ascending=False)
df_relatorio_resumido.to_csv(f"{class_method}_Resumo.csv", sep=";", index=False)
df_relatorio_detalhado.to_csv(f"{class_method}_Detalhe.csv", sep=";", index=False)

# Calculando anomalias

In [8]:
print(f"Média contador emoções: {df_relatorio_detalhado['emotion'].value_counts().mean():.2f}")
print(f"Desvio Padrão contador emoções: {df_relatorio_detalhado['emotion'].value_counts().std():.2f}")

limite_inferior = df_relatorio_detalhado['emotion'].value_counts().mean() - df_relatorio_detalhado['emotion'].value_counts().std()
limite_superior = df_relatorio_detalhado['emotion'].value_counts().mean() + df_relatorio_detalhado['emotion'].value_counts().std()
print(f"Limite Inferior contador emoções: {limite_inferior:.2f}")
print(f"Limite Superior contador emoções: {limite_superior:.2f}")

df_aux = pd.DataFrame(df_relatorio_detalhado['emotion'].value_counts())

Média contador emoções: 887.33
Desvio Padrão contador emoções: 550.21
Limite Inferior contador emoções: 337.12
Limite Superior contador emoções: 1437.54


## Anomalias baseadas em emoções fora do padrão

In [15]:
df_anomalias = pd.concat([df_aux[df_aux['emotion'] <= limite_inferior], df_aux[df_aux['emotion'] >= limite_superior]])

print(f'Total de frames: {total_frames}')

print(f'\nTabela com anomalias detectadas: {total_frames}')
df_anomalias

Total de frames: 3326

Tabela com anomalias detectadas: 3326


,emotion
surprise,171
sad,1687


### Relatório Resumido

In [10]:
df_relatorio_resumido

,type,name,count
95,emotion,sad,1687
96,emotion,fear,1177
97,emotion,neutral,1040
98,emotion,happy,865
99,emotion,angry,384
100,emotion,surprise,171
0,activity,lab coat,776
1,activity,breastplate,533
2,activity,dining table,477
3,activity,mask,401


### Relatório detalhado

In [11]:
df_relatorio_detalhado

,frame_index,total_frames,activity,score,faces_count,emotion
0,0,3326,lab coat,0.406336,4,sad
1,0,3326,lab coat,0.406336,4,fear
2,0,3326,lab coat,0.406336,4,neutral
3,0,3326,lab coat,0.406336,4,angry
4,1,3326,lab coat,0.402197,4,neutral
5,1,3326,lab coat,0.402197,4,sad
6,1,3326,lab coat,0.402197,4,neutral
7,1,3326,lab coat,0.402197,4,angry
8,2,3326,lab coat,0.314377,4,sad
9,2,3326,lab coat,0.314377,4,sad
